In [2]:
import zipfile
from pathlib import Path

import pandas as pd
import geopandas as gpd

alarm = Path("../data-raw/alarm-redist-census-2020")
rdh = Path("../data-raw/rdh")

/Users/tim/projects/reverse-coattails/.direnv/python-3.9.7/lib/python3.9/site-packages/pyproj/__init__.py:91: UserWarning: Valid PROJ data directory not found. Either set the path using the environmental variable PROJ_LIB or with `pyproj.datadir.set_data_dir`.
  warnings.warn(str(err))


In [3]:
list(alarm.iterdir())

[PosixPath('../data-raw/alarm-redist-census-2020/LICENSE.md'),
 PosixPath('../data-raw/alarm-redist-census-2020/vest-2020'),
 PosixPath('../data-raw/alarm-redist-census-2020/crosswalks'),
 PosixPath('../data-raw/alarm-redist-census-2020/R'),
 PosixPath('../data-raw/alarm-redist-census-2020/README.md'),
 PosixPath('../data-raw/alarm-redist-census-2020/census-2020.Rproj'),
 PosixPath('../data-raw/alarm-redist-census-2020/census-vest-2020'),
 PosixPath('../data-raw/alarm-redist-census-2020/.gitignore'),
 PosixPath('../data-raw/alarm-redist-census-2020/.git')]

In [6]:
pd.read_csv(alarm / "vest-2020/pa/pa_2018_2020_vtd.csv").head()

,GEOID20,STATEFP,COUNTYFP,DISTRICT,G18USSDCAS,G18USSRBAR,G18USSLKER,G18USSGGAL,G18GOVDWOL,G18GOVRWAG,G18GOVLKRA,G18GOVGGLO
0,42001000010,42,1,000010,123.0,190.3,5.1,2.1,123.2,192.1,2.1,2.1
1,42001000020,42,1,000020,150.1,176.9,6.0,3.0,159.0,170.9,4.0,2.0
2,42001000030,42,1,000030,74.0,103.0,1.0,2.0,76.0,98.0,3.0,2.0
3,42001000040,42,1,000040,288.6,574.1,14.0,5.0,317.5,553.1,9.0,5.0
4,42001000050,42,1,000050,152.0,231.0,3.0,7.0,168.0,215.0,5.0,2.0


In [9]:
ush = gpd.read_file(rdh / "pa/pa_cong_adopted_2022.zip")

In [10]:
ush.head()

,ID,AREA,DISTRICT,geometry
0,2,67.39,2,"POLYGON ((-74.97398 40.04859, -74.97743 40.045..."
1,3,2836.43,8,"POLYGON ((-75.33468 41.97021, -75.33425 41.970..."
2,5,1183.00,7,"POLYGON ((-75.58984 40.48466, -75.59184 40.485..."
3,8,54.74,3,"POLYGON ((-75.22066 40.00394, -75.21923 40.004..."
4,10,717.30,1,"POLYGON ((-75.19680 40.60858, -75.19671 40.608..."


In [11]:
pal = gpd.read_file(rdh / "pa/pa_sldl_adopted_2022.zip")

In [12]:
pal.head()

,ID,AREA,DISTRICT,MEMBERS,LOCKED,NAME,ADJ_POPULA,ADJ_NH_WHT,ADJ_NH_BLK,ADJ_NH_IND,...,F_ADJ_NH11,F_ADJ_NH12,F_ADJ_NH13,F_ADJ_NH14,F_ADJ_NH15,F_ADJ_NH16,F_ADJ_H18_,DISTRICT_N,DISTRICT_L,geometry
0,1,766.475586,1,1.0,Y,None,65227,42592,10660,97,...,0.143063,0.001576,0.029188,0.000414,0.005258,0.041300,0.067750,1,1|1.83%,"POLYGON ((-80.29889 42.04829, -80.29916 42.048..."
1,2,2.996813,10,1.0,Y,None,61532,15630,35227,121,...,0.527985,0.001954,0.097049,0.000586,0.004534,0.031171,0.045144,10,10|-3.94%,"POLYGON ((-75.23848 39.95349, -75.24045 39.953..."
2,3,309.422150,100,1.0,Y,None,64207,59347,577,60,...,0.009113,0.001008,0.003461,0.000044,0.001818,0.018993,0.031655,100,100|0.24%,"POLYGON ((-75.98506 39.93787, -75.98499 39.937..."
3,4,70.999825,101,1.0,Y,None,65422,43860,1800,63,...,0.026436,0.001067,0.014601,0.000178,0.002331,0.017031,0.217691,101,101|2.14%,"POLYGON ((-76.46631 40.22840, -76.46657 40.229..."
4,5,247.142853,102,1.0,Y,None,65771,59409,666,54,...,0.009238,0.000935,0.012473,0.000468,0.002865,0.020093,0.037127,102,102|2.68%,"POLYGON ((-76.48489 40.33012, -76.48505 40.330..."


In [15]:
print(zipfile.ZipFile(rdh / "pa/pa_sldl_adopted_2022.zip").read("README.txt").decode("ascii"))

2022 Pennsylvania House of Representatives Plan (State Legislative District Lower)

##Redistricting Data Hub (RDH) Retrieval Date
02/08/2022

##Sources
This dataset was retrieved from Pennsylvania Redistricting at: https://www.redistricting.state.pa.us/maps/

##Processing
The RDH did not modify any of the data.

##Additional Notes
Enclosed in this zip file the shapefile file for Pennsylvania's House of Representative Districts ("2022 LRC-Senate-Final.shp") and supporting files, a block equivalency file ("2022 LRC-House-Final.xlsx"), and a report ("LRC House Final.pdf").

To keep track of a state's adopted maps, go to All About Redistricting https://redistricting.lls.edu/
For any additional questions, please email info@redistrictingdatahub.org


In [14]:
zipfile.ZipFile(rdh / "pa/pa_sldl_adopted_2022.zip").filelist

[<ZipInfo filename='2022 LRC-House-Final.csv' compress_type=deflate external_attr=0x20 file_size=8216093 compress_size=1049227>,
 <ZipInfo filename='2022 LRC-House-Final.dbf' compress_type=deflate external_attr=0x20 file_size=114000 compress_size=31912>,
 <ZipInfo filename='2022 LRC-House-Final.prj' compress_type=deflate external_attr=0x20 file_size=165 compress_size=133>,
 <ZipInfo filename='2022 LRC-House-Final.shp' compress_type=deflate external_attr=0x20 file_size=5856768 compress_size=3684234>,
 <ZipInfo filename='2022 LRC-House-Final.shx' compress_type=deflate external_attr=0x20 file_size=1724 compress_size=1319>,
 <ZipInfo filename='LRC House Final.pdf' compress_type=deflate external_attr=0x20 file_size=425021 compress_size=387090>,
 <ZipInfo filename='README.txt' compress_type=deflate external_attr=0x20 file_size=767 compress_size=432>]